In [7]:
import pandas as pd

df = pd.read_csv(r'gen_dataset.csv')
text_data = list(df["text"])
class_data = df["class"].to_numpy()
for i in range(len(class_data)):
    if class_data[i]=='notsarc':
        class_data[i]=0
    else:
        class_data[i]=1
class_data = list(class_data)
train_sarc_text_data = text_data[:int(len(text_data) * 0.8)]
test_sarc_text_data = text_data[int(len(text_data) * 0.8):]

train_sarc_class_data = class_data[:int(len(class_data) * 0.8)]
test_sarc_class_data = class_data[int(len(class_data) * 0.8):]
df

,class,id,text
0,0,1,"If that's true, then Freedom of Speech is doom..."
1,0,2,Neener neener - is it time to go in from the p...
2,0,3,"Just like the plastic gun fear, the armour pie..."
3,0,4,So geology is a religion because we weren't he...
4,0,5,Well done Monty. Mark that up as your first ev...
...,...,...,...
6515,1,6516,depends on when the baby bird died. run alon...
6516,1,6517,"ok, sheesh, to clarify, women who arent aborti..."
6517,1,6518,so.. eh?? hows this sound? will it fly w...
6518,1,6519,"I think we should put to a vote, the right of ..."


In [8]:
df = pd.read_csv(r'irony.csv')
text_data1 = list(df["comment_text"])
class_data1 = df["label"].to_numpy()
for i in range(len(class_data1)):
    if class_data1[i]==1:
        class_data1[i]=1
    else:
        class_data1[i]=0
class_data1 = list(class_data1)    
train_irony_text_data = text_data1[:int(len(text_data1) * 0.8)]
test_irony_text_data = text_data1[int(len(text_data1) * 0.8):]

train_irony_class_data = class_data1[:int(len(class_data1) * 0.8)]
test_irony_class_data = class_data1[int(len(class_data1) * 0.8):]
df

,comment_text,label
0,I suspect atheists are projecting their desire...,0
1,It's funny how the arguments the shills are ma...,0
2,We are truly following the patterns of how the...,0
3,air pressure dropping as altitude goes higher....,0
4,Absolutely. I think we'd be hard pressed to f...,0
...,...,...
1944,This is an interesting point. There are no sh...,0
1945,Maybe you mean how we are to respond to the go...,0
1946,[... What? ](http://tinyurl. com/kw5cpxz),0
1947,Does anybody remember during one of the debate...,0


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel
from torchmetrics.classification import BinaryAccuracy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Load the pre-trained BERT model and tokenizer
model = AutoModel.from_pretrained('roberta-base')
model.to(device)
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Define the classification model
class BertClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        logits = self.fc(pooled_output)
        return logits



# Tokenize the text data
train_encodings = tokenizer(list(train_irony_text_data), padding=True, truncation=True, max_length=128)
test_encodings = tokenizer(list(test_irony_text_data), padding=True, truncation=True, max_length=128)

# Convert the data into PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(list(train_irony_class_data))
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(list(test_irony_class_data))
)


# Define the training parameters
classifier = BertClassifier(model, num_classes=2)
classifier.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=2e-5)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True)
metric = BinaryAccuracy().to(device)


# Train the BERT classifier
classifier.train()
for epoch in range(20):
    print("Epoch ", epoch+1)
    train_losses = []
    train_acc = []
    test_losses = []
    test_acc = []
    for idx, batch in enumerate(train_loader):
        # batch.to(device)
        if(idx % 40 == 0):
            print("-", end="")
        optimizer.zero_grad()
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        loss.backward()
        train_losses.append(loss.cpu().detach().numpy())
        optimizer.step()
        op = torch.argmax(outputs, axis = 1)
        train_acc.append(metric(op, batch[2].to(device)).cpu().detach().numpy())
        
    for idx, batch in enumerate(test_loader):
        # batch.to(device)
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        test_losses.append(loss.cpu().detach().numpy())
        op = torch.argmax(outputs, axis = 1)
        test_acc.append(metric(op, batch[2].to(device)).cpu().detach().numpy())
        
        
    print()
    print("Train Loss: ",sum(train_losses) / len(train_losses), end = "\t")
    print("Test Loss: ",sum(test_losses) / len(test_losses), end = "\n")   
    print("Train Accuracy: ",(sum(train_acc) / len(train_acc)) * 100, end = "% \t")
    print("Test Accuracy: ",(sum(test_acc) / len(test_acc)) * 100, end = "% \n")   

cuda


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch  1
-----
Train Loss:  0.6053553701211245	Test Loss:  0.5648607277140325
Train Accuracy:  71.51098902408893% 	Test Accuracy:  74.40476186421454% 
Epoch  2
-----
Train Loss:  0.5858200664703662	Test Loss:  0.5631560102409247
Train Accuracy:  71.89560440870432% 	Test Accuracy:  74.48979591836735% 
Epoch  3
-----
Train Loss:  0.5894768145604011	Test Loss:  0.5830578232298091
Train Accuracy:  71.78571429008093% 	Test Accuracy:  74.74489795918367% 
Epoch  4
-----
Train Loss:  0.5713978755168426	Test Loss:  0.5541292775650414
Train Accuracy:  72.06959707614703% 	Test Accuracy:  74.14965988421926% 
Epoch  5
-----
Train Loss:  0.5828175121392959	Test Loss:  0.5621761655320927
Train Accuracy:  71.7948717948718% 	Test Accuracy:  74.31972793170384% 
Epoch  6
-----
Train Loss:  0.5672910888225604	Test Loss:  0.5464285298877832
Train Accuracy:  71.92307692307692% 	Test Accuracy:  74.31972793170384% 
Epoch  7
-----
Train Loss:  0.5434634366096595	Test Loss:  0.5195926266665362
Train Accuracy:  

In [10]:
# Load the training data
# train_texts = ["Example sentence 1", "Example sentence 2", ...]
# train_labels = [0, 1, ...]
train_encodings = tokenizer(list(train_sarc_text_data), padding=True, truncation=True, max_length=128)
test_encodings = tokenizer(list(test_sarc_text_data), padding=True, truncation=True, max_length=128)

# Convert the data into PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(list(train_sarc_class_data))
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(list(test_sarc_class_data))
)


# Define the training parameters
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True)
metric = BinaryAccuracy().to(device)


# Train the BERT classifier
classifier.train()
for epoch in range(20):
    print("Epoch ", epoch+1)
    train_losses = []
    train_acc = []
    test_losses = []
    test_acc = []
    for idx, batch in enumerate(train_loader):
        # batch.to(device)
        if(idx % 40 == 0):
            print("-", end="")
        optimizer.zero_grad()
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        loss.backward()
        train_losses.append(loss.cpu().detach().numpy())
        optimizer.step()
        op = torch.argmax(outputs, axis = 1)
        train_acc.append(metric(op, batch[2].to(device)).cpu().detach().numpy())
        
    for idx, batch in enumerate(test_loader):
        # batch.to(device)
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        test_losses.append(loss.cpu().detach().numpy())
        op = torch.argmax(outputs, axis = 1)
        test_acc.append(metric(op, batch[2].to(device)).cpu().detach().numpy())
        
        
    print()
    print("Train Loss: ",sum(train_losses) / len(train_losses), end = "\t")
    print("Test Loss: ",sum(test_losses) / len(test_losses), end = "\n")   
    print("Train Accuracy: ",(sum(train_acc) / len(train_acc)) * 100, end = "% \t")
    print("Test Accuracy: ",(sum(test_acc) / len(test_acc)) * 100, end = "% \n")   
    print()

Epoch  1
-----------------
Train Loss:  0.48625644518096756	Test Loss:  0.4104584828476233
Train Accuracy:  77.74156441717791% 	Test Accuracy:  82.20858895705521% 

Epoch  2
-----------------
Train Loss:  0.36830554340903193	Test Loss:  0.38820645800488857
Train Accuracy:  83.30138036809815% 	Test Accuracy:  82.43865030674846% 

Epoch  3
-----------------
Train Loss:  0.273532012036853	Test Loss:  0.4827343041157247
Train Accuracy:  88.66947852760735% 	Test Accuracy:  81.59509202453987% 

Epoch  4
-----------------
Train Loss:  0.1987866739809593	Test Loss:  0.42800919055664466
Train Accuracy:  92.23542944785275% 	Test Accuracy:  81.44171779141104% 

Epoch  5
-----------------
Train Loss:  0.1288915624460127	Test Loss:  0.5943102032614678
Train Accuracy:  95.43711656441718% 	Test Accuracy:  78.45092024539878% 

Epoch  6
-----------------
Train Loss:  0.0854954016771656	Test Loss:  0.6777588250986843
Train Accuracy:  96.66411042944786% 	Test Accuracy:  81.6717791411043% 

Epoch  7
-----

In [11]:
torch.save(classifier, "RoBERTaTransferIrony.pth")